### Instalación con docker

```console
docker pull docker.elastic.co/elasticsearch/elasticsearch:7.15.2

docker run -p 127.0.0.1:9200:9200 -p 127.0.0.1:9300:9300 -e "discovery.type=single-node" -v /Users/julian/Documents/repos/SQL/data/elasticsearch:/usr/share/elasticsearch/data docker.elastic.co/elasticsearch/elasticsearch:7.15.2
```


In [1]:
from elasticsearch_dsl import connections, Document, Document, Keyword, Integer, Index

In [2]:
es = connections.create_connection(
    alias="default",
    hosts=['http://127.0.0.1:9200'],
    timeout=1000,
    retry_on_timeout=True,
    max_retries=10,
    verify_certs=False
)

In [3]:
class User(Document):
    name = Keyword()
    address = Keyword()
    age = Integer()
    
    class Index:
        name = 'users'
        settings = {
            'number_of_shards': 1,
            'number_of_replicas': 0
        }

In [4]:
User.init()

ConnectionError: ConnectionError(<urllib3.connection.HTTPConnection object at 0x0000015665983D40>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it) caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x0000015665983D40>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it)

In [5]:
# Creación de usuario
user1 = User(name='Pedro Martinez', address='Alsina 1234', age=45)
user2 = User(name='Marina Gomez', address='Libertad 4321', age=83)

In [6]:
# Guardo Usuario
user1.save()
user2.save()

'created'

In [7]:
User.search().count()

2

In [8]:
users = User.search().filter().execute()

In [9]:
users.timed_out

False

In [10]:
for u in users:
    print(u.to_dict(), u.meta.id)

{'name': 'Pedro Martinez', 'address': 'Alsina 1234', 'age': 45} kvBw4IkBs4b4bRwR2Alj
{'name': 'Marina Gomez', 'address': 'Libertad 4321', 'age': 83} k_Bw4IkBs4b4bRwR2AmG


In [11]:
u.meta.to_dict()

{'index': 'users',
 'id': 'k_Bw4IkBs4b4bRwR2AmG',
 'score': 0.0,
 'doc_type': '_doc'}

In [12]:
Index('users').get_mapping()

{'users': {'mappings': {'properties': {'address': {'type': 'keyword'},
    'age': {'type': 'integer'},
    'name': {'type': 'keyword'}}}}}

In [13]:
# Si no lo tiene configurado lo infiere 'gender'
user3 = User(name='Maria Diaz', address='Guayra 2323', age=21, gender='M')

In [14]:
user3.save()

'created'

In [15]:
for u in users:
    print(u.to_dict(), u.meta.id)

{'name': 'Pedro Martinez', 'address': 'Alsina 1234', 'age': 45} kvBw4IkBs4b4bRwR2Alj
{'name': 'Marina Gomez', 'address': 'Libertad 4321', 'age': 83} k_Bw4IkBs4b4bRwR2AmG


In [16]:
Index('users').get_mapping()

{'users': {'mappings': {'properties': {'address': {'type': 'keyword'},
    'age': {'type': 'integer'},
    'gender': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'name': {'type': 'keyword'}}}}}